# Linear Regression

In [ ]:
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt

print(tf.__version__)

## Generate Demo Data

In [ ]:
#  generation some house sizes between 1000 and 3500 (typical sq ft of house)
num_house = 160
np.random.seed(58)
house_size = np.random.randint(low=1000, high=3500, size=num_house )

# Generate house prices from house size with a random noise added.
np.random.seed(58)
house_price = house_size * 100.0 + np.random.randint(low=20000, high=70000, size=num_house)  

# Plot generated house and size 
plt.plot(house_size, house_price, "bx")  # bx = blue x
plt.ylabel("Price")
plt.xlabel("Size")
plt.show()

## Split Data Into Training and Test

In [ ]:
# you need to normalize values to prevent under/overflows.
def normalize(array):
    return (array - array.mean()) / array.std()

# define number of training samples, 0.7 = 70%.  We can take the first 70% since the values are randomized
num_train_samples = int(math.floor(num_house * 0.7))

# define training data
train_house_size = np.asarray(house_size[:num_train_samples])
train_price = np.asanyarray(house_price[:num_train_samples:])

train_house_size_norm = normalize(train_house_size)
train_price_norm = normalize(train_price)

# define test data
test_house_size = np.array(house_size[num_train_samples:])
test_house_price = np.array(house_price[num_train_samples:])

test_house_size_norm = normalize(test_house_size)
test_house_price_norm = normalize(test_house_price)

# Plot the graph
plt.rcParams["figure.figsize"] = (10,8)
plt.figure()
plt.ylabel("Price")
plt.xlabel("Size (sq.ft)")
plt.plot(train_house_size, train_price, 'go', label='Training data')
plt.plot(test_house_size, test_house_price, 'mo', label='Testing data')

plt.legend(loc='upper left')
plt.show()

## Define Model

In [ ]:
#  Set up the TensorFlow placeholders that get updated as we descend down the gradient
tf_house_size = tf.placeholder("float", name="house_size") # x / features
tf_price = tf.placeholder("float", name="price") # y_ / labels

# Define the variables holding the size_factor and price we set during training.  
# We initialize them to some random values based on the normal distribution.
tf_size_factor = tf.Variable(np.random.randn(), name="size_factor") # weights
tf_price_offset = tf.Variable(np.random.randn(), name="price_offset") # bias

### Hypothesis
Define the operations for the predicting values:
![Hypothesis](img/lin_reg_hypothesis.png)
Notice, the use of the tensorflow add and multiply functions. These add the operations to the computation graph, AND the tensorflow methods understand how to deal with Tensors.  Therefore do not try to use numpy or other library methods.

In [ ]:
tf_price_pred = # ADD YOUR CODE HERE

### Loss Function
Define the Loss Function (how much error) - Mean squared error
![MeanSquaredError1](img/mean_squared_error.png)

In [ ]:
loss = # ADD YOUR CODE HERE

### Gradient Descent
![MeanSquaredError1](img/gradient_descent.png)

In [ ]:
# Optimizer learning rate.  The size of the steps down the gradient
learning_rate = 0.1

# 4. define a Gradient descent optimizer that will minimize the loss defined in the operation "cost".
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

## Train Model

In [ ]:
# Initializing the variables
init = tf.global_variables_initializer()

size_factor = None
price_offset = None

# Launch the graph in the session
with tf.Session() as sess:
    sess.run(init)

    # set how often to display training progress and number of training iterations
    display_every = 2
    num_training_iter = 50

    # keep iterating the training data
    for iteration in range(num_training_iter):

        # Fit all training data
        for (x, y) in zip(train_house_size_norm, train_price_norm):
            sess.run(train, feed_dict={tf_house_size: x, tf_price: y})

        # Display current status
        if (iteration + 1) % display_every == 0:
            c = sess.run(loss, feed_dict={tf_house_size: train_house_size_norm, tf_price:train_price_norm})
            print("iteration #:", '%04d' % (iteration + 1), "cost=", "{:.9f}".format(c), \
                "size_factor=", sess.run(tf_size_factor), "price_offset=", sess.run(tf_price_offset))

    print("Optimization Finished!")
    training_cost = sess.run(loss, feed_dict={tf_house_size: train_house_size_norm, tf_price: train_price_norm})
    size_factor = sess.run(tf_size_factor)
    price_offset = sess.run(tf_price_offset)
    print("Trained cost=", training_cost, "size_factor=", size_factor, "price_offset=", price_offset, '\n')
    
    # Plot of training and test data, and learned regression
    
    # get values used to normalized data so we can denormalize data back to its original scale
    train_house_size_mean = train_house_size.mean()
    train_house_size_std = train_house_size.std()

    train_price_mean = train_price.mean()
    train_price_std = train_price.std()
    
    # Plot the graph
    plt.rcParams["figure.figsize"] = (10,8)
    plt.figure()
    plt.ylabel("Price")
    plt.xlabel("Size (sq.ft)")
    plt.plot(train_house_size, train_price, 'go', label='Training data')
    plt.plot(test_house_size, test_house_price, 'mo', label='Testing data')
    plt.plot(train_house_size_norm * train_house_size_std + train_house_size_mean,
             (size_factor * train_house_size_norm + price_offset) * train_price_std + train_price_mean,
             label='Learned Regression')
 
    plt.legend(loc='upper left')
    plt.show()